
# Chapter 3: OSTEP 청크 임베딩 생성

chapter.2에서 생성된 OSTEP 청크를 임베딩으로 변환해 벡터 검색 준비를 합니다.

## 📚 학습 목표
- 청크 JSON을 임베딩 입력 텍스트로 안전히 변환(JSON 문자열화 전략)
- SentenceTransformer로 고속 임베딩 생성 및 정규화 벡터 이해
- 파일 저장/형식(NumPy NPY)과 이후 검색 인덱스 연계를 위한 산출물 관리

## 📋 실습 구성
- 1️⃣ 환경 설정: 패키지 설치/디바이스 확인/경로 설정
- 2️⃣ 데이터 로드: 청크 JSON 로드 및 샘플 구조 확인
- 3️⃣ 임베딩 생성: 정규화 포함 일괄 임베딩
- 4️⃣ 결과 저장: `.npy`로 저장, 크기/형식 확인

> ⚠️ 임베딩 생성 시 메모리 사용량에 유의하세요(GPU/CPU 환경 차이 존재).

---
## 1️⃣ Google Colab 환경 설정

이 노트북은 **Google Colab에서 GPU를 사용**하여 실행하도록 설계되었습니다.

### 📌 실행 전 준비사항
1. **런타임 유형 설정**: 메뉴에서 `런타임` → `런타임 유형 변경` → `GPU` 선택
2. **첫 번째 코드 셀 실행**: Google Drive 마운트 및 필수 패키지 자동 설치

> ⚠️ **중요**: GPU를 사용하면 임베딩 생성 속도가 대폭 향상됩니다.


In [1]:
# ========================================
# Google Colab 환경 설정
# ========================================
from google.colab import drive
import os
import warnings

# Google Drive 마운트
drive.mount('/content/drive')

# 필요 패키지 설치
!pip -q install sentence-transformers torch numpy

# 경고 억제
warnings.filterwarnings('ignore')

# 라이브러리 임포트
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import json
from pathlib import Path

# GPU 자동 감지
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Using device: {DEVICE}")
if DEVICE == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# 경로 설정
BASE_DIR = "/content/drive/MyDrive/ostep_rag"
DATA_DIR = os.path.join(BASE_DIR, "data")
CHUNK_FILE = os.path.join(DATA_DIR, "chunk", "ostep_tok400_ov20.json")
OUTPUT_DIR = os.path.join(DATA_DIR, "vector")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 모델 설정
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
EMBEDDING_DIM = 384

# 모델 로드
print(f"\nModel: {MODEL_NAME}")
model = SentenceTransformer(MODEL_NAME, device=DEVICE)
print("✅ Model loaded successfully!")


Model: sentence-transformers/all-MiniLM-L6-v2
Device: cpu
Model loaded successfully!


---
## 2️⃣ 데이터 로드

이 셀에서는 chapter.2에서 생성한 청크 데이터를 로드합니다.

**주요 내용:**
- JSON 파일에서 청크 데이터 읽기
- 청크 개수 및 구조 확인
- 텍스트 추출

**실행 결과:**
- 청크 데이터가 성공적으로 로드되고 개수가 출력됩니다.


In [2]:
# 청크 데이터 로드
print(f"Loading chunks from: {CHUNK_FILE}")
with open(CHUNK_FILE, 'r', encoding='utf-8') as f:
    chunks = json.load(f)

print(f"Loaded {len(chunks)} chunks")

# 청크 구조 확인
if chunks:
    print("\nSample chunk structure:")
    sample = chunks[0]
    for key, value in sample.items():
        if key == 'text':
            print(f"  {key}: {value[:100]}..." if len(str(value)) > 100 else f"  {key}: {value}")
        else:
            print(f"  {key}: {value}")

# 청크 전체를 JSON 문자열로 변환하여 임베딩
# 이렇게 하면 모든 필드(chunk_id, chapter_id, chapter_title, subsection_id, subsection_title, text)가 포함됨
def chunk_to_embedding_text(chunk):
    """청크 전체를 JSON 문자열로 변환하여 임베딩 텍스트 생성"""
    return json.dumps(chunk, ensure_ascii=False)

texts = [chunk_to_embedding_text(chunk) for chunk in chunks]
print(f"\nConverted {len(texts)} chunks to JSON strings for embedding")

# 샘플 출력
print("\nSample embedding text (first 200 chars):")
print(f"  {texts[0][:200]}...")


Loading chunks from: ../data/chunk/ostep_tok400_ov20.json
Loaded 991 chunks

Sample chunk structure:
  chunk_id: ch_1__0000
  chapter_id: ch_1
  chapter_title: A Dialogue on the Book
  subsection_id: None
  subsection_title: None
  text: A Dialogue on the Book Professor: Welcome to this book! It’s called Operating Systems in Three Easy ...

Converted 991 chunks to JSON strings for embedding

Sample embedding text (first 200 chars):
  {"chunk_id": "ch_1__0000", "chapter_id": "ch_1", "chapter_title": "A Dialogue on the Book", "subsection_id": null, "subsection_title": null, "text": "A Dialogue on the Book Professor: Welcome to this ...


---
## 3️⃣ 임베딩 생성

이 셀에서는 SentenceTransformer를 사용하여 모든 청크를 임베딩합니다.

**주요 내용:**
- 991개 청크를 한번에 임베딩 생성
- 각 청크는 JSON 문자열로 변환되어 전체 정보가 포함됨
- 정규화된 벡터 생성 (코사인 유사도 계산 최적화)
- 임베딩 차원 확인

**실행 결과:**
- 모든 청크의 임베딩이 생성되고 shape가 출력됩니다.


In [3]:
print("Generating embeddings...")
print(f"Processing {len(texts)} chunks...")

# 임베딩 생성 (정규화된 벡터, float32)
embeddings = model.encode(
    texts,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
)

print(f"\nEmbeddings generated successfully!")
print(f"Embedding shape: {embeddings.shape}")
print(f"Embedding dtype: {embeddings.dtype}")
print(f"First embedding sample (first 5 values): {embeddings[0][:5]}")


Generating embeddings...
Processing 991 chunks...


Batches:   0%|          | 0/31 [00:00<?, ?it/s]


Embeddings generated successfully!
Embedding shape: (991, 384)
Embedding dtype: float32
First embedding sample (first 5 values): [-0.1130181   0.00220718 -0.01913215  0.01023912 -0.03593146]


---
## 4️⃣ 결과 저장

이 셀에서는 생성된 임베딩를 파일로 저장합니다.

**주요 내용:**
- NumPy 배열(.npy)로 임베딩 저장
- FAISS/ChromaDB 호환 형식

**실행 결과:**
- 두 개의 파일이 생성되고 저장 경로가 출력됩니다.


In [6]:
# 출력 파일 경로 설정
embeddings_file = Path(OUTPUT_DIR) / "ostep_tok400_ov20_embeddings.npy"

# 임베딩 저장
print(f"Saving embeddings to: {embeddings_file}")
np.save(embeddings_file, embeddings.astype(np.float32))
print(f"✓ Embeddings saved: {embeddings.shape}")

# 파일 크기 확인
emb_size = embeddings_file.stat().st_size / (1024 * 1024)  # MB

print(f"\nFile sizes:")
print(f"  Embeddings: {emb_size:.2f} MB")


Saving embeddings to: ../data/vector/ostep_tok400_ov20_embeddings.npy
✓ Embeddings saved: (991, 384)

File sizes:
  Embeddings: 1.45 MB
